In [136]:
from datasets import load_dataset
import sentencepiece as spm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import nltk
from nltk.tokenize import sent_tokenize
import random
from datasets import concatenate_datasets
from datasets import Dataset

## Loading the Datasets

In [37]:
# Load the dataset
#dataset = load_dataset("t1annnnn/Chinese_sentimentAnalyze")
dataset = load_dataset("sepidmnorozy/Chinese_sentiment")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset_2 = load_dataset("t1annnnn/Chinese_sentimentAnalyze")

In [46]:
test_dataset_2 = dataset["test"]

In [95]:
print(test_dataset_2[1])

{'label': 1, 'text': '故事 情節 很好 很 唯美 我的 很多 同學 都說 很好 看 ， 但是 我認為 一本 好的 書 裡面 的 東西 也 很多 主題 有了 ， 但是 次 要 的 也 很 重要 的 ， 換 位 想 人的 一生 要有 很多 東西 不只是 愛情 。 我的 一位 同學 說 結尾 少點 東西 ， 我 個人 也是 這樣 認為 我 覺得 少了 女主角 的 親情 。 你說 也 感覺到 太 單調 ， 我 很 期待 那 本 《 代 孕 媽媽 》 我 看了 前面 我 覺得 比 你 前面 寫的 都好 裡面 有 東西 了 ， 內容 豐富 了 人物 真實 了 ， 這 說明 你的 筆鋒 成熟 了 。 加油 哦 ！'}


In [87]:
import requests
from bs4 import BeautifulSoup

# Specify the URL
urls = ["https://zh.wikipedia.org/wiki/%E4%B8%AD%E5%9C%8B", "https://zh.wikipedia.org/wiki/%E4%B8%AD%E5%8D%8E%E4%BA%BA%E6%B0%91%E5%85%B1%E5%92%8C%E5%9B%BD"]
#url = "https://zh.wikipedia.org/wiki/%E4%B8%AD%E5%9C%8B"
#url = "https://zh.wikipedia.org/wiki/%E4%B8%AD%E5%8D%8E%E4%BA%BA%E6%B0%91%E5%85%B1%E5%92%8C%E5%9B%BD"
articles_content = []
for url in urls: #fill articles_content with content from url's
    response = requests.get(url)
    # Adjust encoding based on the site's specific encoding
    response.encoding = 'UTF-8'
    soup = BeautifulSoup(response.text, "html.parser")
    # Adjust the selection based on the content structure of the site
    article_content = soup.find("div", class_="mw-page-container-inner")  # Use the correct tag and class_ or id
    if article_content:
        articles_content.append(article_content.get_text())

In [116]:
#creating a data frame of sentences
nltk.download('punkt')
all_sentences = []
for content in articles_content:
    sentences = sent_tokenize(content)
    all_sentences.extend(sentences)
df = pd.DataFrame(all_sentences, columns=['text'])
df.reset_index(drop=True, inplace=True)
print(df.head())
#df.to_csv("neutral_chinese_sentiment.csv", index=False)

                                                text
0  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n目录\n移至侧栏\n...
1                                       《中原文化与民族复兴》.
2                                           河南人民出版社.
3                                         2010: 第7頁.
4                                ISBN 9787215072756.


[nltk_data] Downloading package punkt to C:\Users\Dell
[nltk_data]     3520\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [143]:
#turning dataframe into a dictionary
#print(df.columns)
#df.columns = ['text']  
df['data'] = df['text'].apply(lambda x: {'label': 2, 'text': x}) #every neutral will be referenced with a 2
neutral_data = df['data'].tolist()
total_length = len(neutral_data)
split_index = int(total_length * 0.75)
neutral_training_data = neutral_data[:split_index]
data_dict = {key: [dic[key] for dic in neutral_training_data] for key in neutral_training_data[0]}
neutral_training_dataset = Dataset.from_dict(data_dict)
combined_training_dataset = concatenate_datasets([train_dataset,neutral_training_dataset])
train_dataset = combined_training_dataset.shuffle(seed = 42)

neutral_testing_data = neutral_data[split_index:]
data_dict = {key: [dic[key] for dic in neutral_testing_data] for key in neutral_testing_data[0]}
neutral_testing_dataset = Dataset.from_dict(data_dict)
combined_testing_dataset = concatenate_datasets([test_dataset,neutral_testing_dataset])
test_dataset = combined_testing_dataset.shuffle(seed=42)

## Tokenizing and Vectorizing

In [144]:
# Create and write to chinese_data.txt for SnetencePiece training
with open("chinese_data.txt", "w", encoding="utf-8") as file:
    for example in train_dataset:
        file.write(example["text"] + "\n")

#Training the data set model:
spm.SentencePieceTrainer.Train('--input=chinese_data.txt --model_prefix=chinese_model --vocab_size=8000 --character_coverage=0.9995 --model_type=bpe')

# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor()
sp.Load("chinese_model.model")

# Tokenize all texts in the training dataset
tokenized_texts = [sp.EncodeAsPieces(text) for text in train_dataset['text']]
#for testing
tokenized_test_texts = [sp.EncodeAsPieces(text) for text in test_dataset['text']]

# Converting token lists back to strings
joined_texts = [' '.join(tokens) for tokens in tokenized_texts]
# Joining the tokens into strings for each document in the test set
joined_test_texts = [' '.join(tokens) for tokens in tokenized_test_texts]

# Creating a TfidfVectorizer object



In [49]:
tokenized_test_texts_2 = [sp.EncodeAsPieces(text) for text in test_dataset_2['text']]
joined_test_texts_2 = [' '.join(tokens) for tokens in tokenized_test_texts_2]
vectorizer = TfidfVectorizer(ngram_range = (1,3))

## Creating Feature variables

In [145]:
# Fitting the model and transforming the text data into TF-IDF vectors
X_train = vectorizer.fit_transform(joined_texts)
X_test = vectorizer.transform(joined_test_texts)
X_test_2 = vectorizer.transform(joined_test_texts_2)

In [6]:
'''

# Calculate the sum of TF-IDF scores for each feature (word)
sum_tfidf = np.array(X.sum(axis=0)).flatten()
#print(sum_tfidf)
# Calculate the average by dividing by the number of documents
average_tfidf = sum_tfidf / X.shape[0]

# Get feature names to map indices to words
feature_names = vectorizer.get_feature_names_out()

# Map from feature name to average tf-idf score
word_to_avg_score = {feature_names[i]: average_tfidf[i] for i in range(len(feature_names))}
'''

'\n\n# Calculate the sum of TF-IDF scores for each feature (word)\nsum_tfidf = np.array(X.sum(axis=0)).flatten()\n#print(sum_tfidf)\n# Calculate the average by dividing by the number of documents\naverage_tfidf = sum_tfidf / X.shape[0]\n\n# Get feature names to map indices to words\nfeature_names = vectorizer.get_feature_names_out()\n\n# Map from feature name to average tf-idf score\nword_to_avg_score = {feature_names[i]: average_tfidf[i] for i in range(len(feature_names))}\n'

## Creating Target Variables

In [146]:
y_train = [data['label'] for data in train_dataset]
y_test = [data['label'] for data in test_dataset]
y_test_2 = [data['label'] for data in test_dataset_2]

## Creating and Testing LogisticRegression model

In [149]:
default_model = LogisticRegression()

# Train the model
default_model.fit(X_train, y_train)
# Evaluate the model
default_predictions = default_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, default_predictions))
print(classification_report(y_test, default_predictions))

Accuracy: 0.9549041713641488
              precision    recall  f1-score   support

           0       0.91      0.86      0.88      1921
           1       0.92      0.94      0.93      2975
           2       0.99      1.00      0.99      5748

    accuracy                           0.95     10644
   macro avg       0.94      0.93      0.93     10644
weighted avg       0.95      0.95      0.95     10644



In [9]:
'''model_1 = LogisticRegression(C=0.01, penalty = 'l1', solver = 'saga') #c=0.01

model_1.fit(X_train, y_train)
# Evaluate the model
model_1_predictions = model_1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, model_1_predictions))
print(classification_report(y_test, model_1_predictions))'''

'model_1 = LogisticRegression(C=0.01, penalty = \'l1\', solver = \'saga\') #c=0.01\n\nmodel_1.fit(X_train, y_train)\n# Evaluate the model\nmodel_1_predictions = model_1.predict(X_test)\nprint("Accuracy:", accuracy_score(y_test, model_1_predictions))\nprint(classification_report(y_test, model_1_predictions))'

In [10]:
'''model_1 = LogisticRegression(C=0.1, penalty = 'l1', solver = 'saga')#c=0.1

model_1.fit(X_train, y_train)
# Evaluate the model
model_1_predictions = model_1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, model_1_predictions))
print(classification_report(y_test, model_1_predictions))'''

'model_1 = LogisticRegression(C=0.1, penalty = \'l1\', solver = \'saga\')#c=0.1\n\nmodel_1.fit(X_train, y_train)\n# Evaluate the model\nmodel_1_predictions = model_1.predict(X_test)\nprint("Accuracy:", accuracy_score(y_test, model_1_predictions))\nprint(classification_report(y_test, model_1_predictions))'

In [20]:
model_1 = LogisticRegression(C=1, penalty = 'l2', solver = 'saga') #c=1

model_1.fit(X_train, y_train)
# Evaluate the model
model_1_predictions = model_1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, model_1_predictions))
print(classification_report(y_test, model_1_predictions))

Accuracy: 0.7656617606828254
              precision    recall  f1-score   support

           0       0.75      0.78      0.77      9089
           1       0.78      0.75      0.76      9188

    accuracy                           0.77     18277
   macro avg       0.77      0.77      0.77     18277
weighted avg       0.77      0.77      0.77     18277



In [12]:
model_1 = LogisticRegression(C=1, penalty = 'l1', solver = 'liblinear') #c=1

model_1.fit(X_train, y_train)
# Evaluate the model
model_1_predictions = model_1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, model_1_predictions))
print(classification_report(y_test, model_1_predictions))

Accuracy: 0.7644033484707556
              precision    recall  f1-score   support

           0       0.75      0.78      0.77      9089
           1       0.78      0.75      0.76      9188

    accuracy                           0.76     18277
   macro avg       0.76      0.76      0.76     18277
weighted avg       0.76      0.76      0.76     18277



In [13]:
logistic_regression_model = LogisticRegression()

'''param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']  # solvers that support l1 penalty
}

print("gridsearch")
# Setup the grid search
grid_search = GridSearchCV(logistic_regression_model, param_grid, cv=5, scoring='accuracy')

print("fit")
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)'''

'param_grid = {\n    \'C\': [0.01, 0.1, 1, 10, 100],\n    \'penalty\': [\'l1\', \'l2\'],\n    \'solver\': [\'liblinear\', \'saga\']  # solvers that support l1 penalty\n}\n\nprint("gridsearch")\n# Setup the grid search\ngrid_search = GridSearchCV(logistic_regression_model, param_grid, cv=5, scoring=\'accuracy\')\n\nprint("fit")\n# Fit the grid search to the data\ngrid_search.fit(X_train, y_train)\n\n# Best parameters and best score\nprint("Best Parameters:", grid_search.best_params_)\nprint("Best Score:", grid_search.best_score_)'

In [14]:
logistic_regression_model.fit(X_train, y_train)

y_pred = logistic_regression_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.7650599113640094
              precision    recall  f1-score   support

           0       0.76      0.78      0.77      9089
           1       0.78      0.75      0.76      9188

    accuracy                           0.77     18277
   macro avg       0.77      0.77      0.77     18277
weighted avg       0.77      0.77      0.77     18277



In [15]:
incorrect_texts = []
correct_texts = []
incorrect_predictions = []
correct_labels = []

for i in range(len(y_test)):
    if y_pred[i] != y_test[i]:
        incorrect_texts.append(joined_test_texts[i])
        incorrect_predictions.append(y_pred[i])
        correct_labels.append(y_test[i])
    else:
        correct_texts.append(joined_test_texts[i])

average_incorrect_length = []
sum_incorrect_length = 0
for j in range(len(incorrect_texts)):
    sum_incorrect_length = sum_incorrect_length + len(incorrect_texts[j])

average_incorrect_length = sum_incorrect_length/j

average_correct_length = []
sum_correct_length = 0
for h in range(len(correct_texts)):
    sum_correct_length = sum_correct_length + len(correct_texts[h])

average_correct_length = sum_correct_length/h

print(f"Average correct length: {average_correct_length}")
print(f"Average incorrect length: {average_incorrect_length}")
print(incorrect_texts[5])
print(len(incorrect_texts[5]))

Average correct length: 76.89114575883279
Average incorrect length: 73.64779874213836
▁回复 靖 添 尧 俺 家 员工 没有 退 役 潜 水 员 ▁ 靖 添 尧 为 你的 员工 负责 必须 米 有 ▁ 嗨 辣 老 孙 今儿 打 死 也没有 ▁张 玮 倩 孙 总 有 外 卖 么 ▁ 嗨 辣 老 孙 ▁回复 张 新 民 各种 鱼 各种 丸 儿 各种 贝 壳 儿 ▁张 新 民 都是 什么 货 色 啊
155


In [16]:
'''sgd_classifier = SGDClassifier(loss='hinge',  # Use 'hinge' for linear SVM, 'log' for logistic regression
                               max_iter=1000,
                               tol=1e-3,
                               random_state=42)
sgd_classifier.fit(X_train, y_train)

y_pred = sgd_classifier.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))'''

'sgd_classifier = SGDClassifier(loss=\'hinge\',  # Use \'hinge\' for linear SVM, \'log\' for logistic regression\n                               max_iter=1000,\n                               tol=1e-3,\n                               random_state=42)\nsgd_classifier.fit(X_train, y_train)\n\ny_pred = sgd_classifier.predict(X_test)\n\nprint("Accuracy:", accuracy_score(y_test, y_pred))\nprint(classification_report(y_test, y_pred))'

In [17]:
'''svm_classifier = SVC(kernel='linear') 
svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))'''

'''linear_svc_model = LinearSVC(max_iter=1000) 
linear_svc_model.fit(X_train, y_train)
y_pred = linear_svc_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))'''

'linear_svc_model = LinearSVC(max_iter=1000) \nlinear_svc_model.fit(X_train, y_train)\ny_pred = linear_svc_model.predict(X_test)\n\nprint("Accuracy:", accuracy_score(y_test, y_pred))\nprint(classification_report(y_test, y_pred))'

## Naive Bayes

In [18]:
'''naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train, y_train)
y_pred_NB = naive_bayes_model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report

# Calculate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred_NB))

print(classification_report(y_test, y_pred_NB))
'''

'naive_bayes_model = MultinomialNB()\nnaive_bayes_model.fit(X_train, y_train)\ny_pred_NB = naive_bayes_model.predict(X_test)\nfrom sklearn.metrics import accuracy_score, classification_report\n\n# Calculate accuracy\nprint("Accuracy:", accuracy_score(y_test, y_pred_NB))\n\nprint(classification_report(y_test, y_pred_NB))\n'

## Random Forest

In [19]:
'''random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)  
random_forest_model.fit(X_train, y_train)
y_pred_RF = random_forest_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))'''

'random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)  \nrandom_forest_model.fit(X_train, y_train)\ny_pred_RF = random_forest_model.predict(X_test)\nprint("Accuracy:", accuracy_score(y_test, y_pred))\n\nprint(classification_report(y_test, y_pred))'

## SGDClassifier

In [43]:
#sgd_classifier = SGDClassifier(alpha = 0.0001, loss = 'log_loss', learning_rate = 'optimal', random_state=42, penalty = 'l2')
sgd_classifier = SGDClassifier()

sgd_classifier.fit(X_train, y_train)
y_pred = sgd_classifier.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9466911764705882
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      1921
           1       0.95      0.96      0.96      2975

    accuracy                           0.95      4896
   macro avg       0.95      0.94      0.94      4896
weighted avg       0.95      0.95      0.95      4896



Hyperparameter searching